In [3]:
import requests
import speech_recognition as sr
from gtts import gTTS
import os
from playsound import playsound


In [5]:

# Set Ollama endpoint and model details
OLLAMA_URL = "http://localhost:11434/api/chat"  # Default Ollama API URL
MODEL_NAME = "QuantFactory/Llama-3-8B-Instruct-Finance-RAG-GGUF:Q3_K_S"

def get_voice_input():
    """Capture and transcribe voice input using a microphone."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for your query...")
        recognizer.adjust_for_ambient_noise(source)
        try:
            audio = recognizer.listen(source)
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            return None
        except sr.RequestError as e:
            print(f"Speech recognition error: {e}")
            return None

def query_ollama(model_name, prompt):
    """Send the prompt to Ollama and get the response."""
    payload = {
        "model": model_name,
        "prompt": prompt,
    }
    try:
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        return response.json().get("response", "")
    except requests.exceptions.RequestException as e:
        print(f"Error querying Ollama: {e}")
        return None

def text_to_speech(text, filename="response.mp3"):
    """Convert text to speech and save it to a file."""
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    playsound(filename)
    os.remove(filename)

def main():
    """Main function to handle voice interaction."""
    while True:
        print("\nSay 'exit' to quit.")
        user_input = get_voice_input()
        if user_input is None:
            continue
        if user_input.lower() == "exit":
            print("Exiting. Goodbye!")
            break

        # Query the model
        response = query_ollama(MODEL_NAME, user_input)
        if response:
            print(f"Model response: {response}")

            # Convert response to speech
            text_to_speech(response)
        else:
            print("No response from the model.")


In [ ]:

if __name__ == "__main__":
    main()
